# Updating root IDs based on a nucleus table

In this example, we'll look at how to take a set of root IDs which (at some point),
represented neurons, and update them to the most current version for a segmentation
at a later time.

For this example, it is important to understand that a root ID is a unique identifier
for an object in the segmentation _at a given point in time_, so if the segmentation
changes, the root ID may no longer be valid. A common pattern to keep track of neurons
given this dynamic segmentation, then, is to keep track of nucleus IDs. Nucleus IDs are
tied to a spatial point in the segmentation, so at any point in time, we can simply look
up the root ID of that point to identify "this neuron." It isn't foolproof (say, if someone
removes the nucleus by accident while proofreading), but for the most part it will be
what you want.


In [2]:
import caveclient as cc

client = cc.CAVEclient("minnie65_phase3_v1")

First, we'll select some root IDs


In [13]:
import pandas as pd
import numpy as np

roots = np.array(
    [
        864691135214134328,
        864691135387648385,
        864691136388711031,
        864691134886015738,
        864691134886016762,
        864691134948652540,
        864691135013445270,
        864691135082074103,
        864691135082074359,
        864691135082840567,
    ]
)

neuron_manifest = pd.DataFrame()
neuron_manifest["working_root_id"] = roots
neuron_manifest

,working_root_id
0,864691135214134328
1,864691135387648385
2,864691136388711031
3,864691134886015738
4,864691134886016762
5,864691134948652540
6,864691135013445270
7,864691135082074103
8,864691135082074359
9,864691135082840567


We can check the chunkedgraph to see if these are the latest root IDs for these objects:


In [14]:
is_current_mask = client.chunkedgraph.is_latest_roots(roots)
neuron_manifest["is_current"] = is_current_mask
neuron_manifest

,working_root_id,is_current
0,864691135214134328,False
1,864691135387648385,False
2,864691136388711031,False
3,864691134886015738,True
4,864691134886016762,True
5,864691134948652540,True
6,864691135013445270,True
7,864691135082074103,True
8,864691135082074359,True
9,864691135082840567,True


Next we'll define the nucleus table we want to use to update the root IDs.


In [15]:
NUCLEUS_TABLE = "nucleus_detection_v0"

Now, we can loop through these outdated root IDs, and for each, look up which objects
in the segmentation contain part of our previous neuron (using `get_latest_roots`). We
can then look up the nucleus ID for each of these objects, and use that to update our
root ID. 

Note that if there is more than one related object in the segmentation which is 
in the nucleus table, we'll reach an ambiguity, since we won't know which one to use.
A future example will cover how to handle this case.


In [19]:
outdated_roots = roots[~is_current_mask]
root_map = dict(zip(roots[is_current_mask], roots[is_current_mask]))
for outdated_root in outdated_roots:
    latest_roots = client.chunkedgraph.get_latest_roots(outdated_root)
    possible_nucs = client.materialize.query_table(
        NUCLEUS_TABLE, filter_in_dict={"pt_root_id": latest_roots}
    )
    if len(possible_nucs) == 1:
        root_map[outdated_root] = possible_nucs.iloc[0]["pt_root_id"]
    else:
        print(f"Multiple nucleus-bound roots for {outdated_root}")
        root_map[outdated_root] = None

updated_roots = pd.Series(
    [root_map[root] for root in roots], dtype="Int64"
)
neuron_manifest["current_root_id"] = updated_roots
neuron_manifest

,working_root_id,is_current,current_root_id
0,864691135214134328,False,864691136117421732
1,864691135387648385,False,864691135938404612
2,864691136388711031,False,864691135404792046
3,864691134886015738,True,864691134886015738
4,864691134886016762,True,864691134886016762
5,864691134948652540,True,864691134948652540
6,864691135013445270,True,864691135013445270
7,864691135082074103,True,864691135082074103
8,864691135082074359,True,864691135082074359
9,864691135082840567,True,864691135082840567
